# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver import Solver
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

### Problem definition

Initial point, target, UAV airspeed

In [2]:
# Scale factor
sf = 1e6

# Initial point
x_init = sf * np.array((0., 0.))

# Target
x_target = sf * np.array((1., 0.))

# Windfield boundaries
bl = sf * np.array((-0.1, -1.))
tr = sf * np.array((1.1, 1.))

# UAV airspeed in m/s
v_a = 23.

case_name = 'vortex_1'

Set up the problem

In [3]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)

# Problem coordinates
# coords = COORD_GCS
coords = COORD_CARTESIAN

# Problem wind data
from mermoz.wind import RankineVortexWind, PointSymWind
# total_wind = RankineVortexWind(sf * 0.5, sf * 0.3, sf * v_a * 3., sf * 0.2)
gamma = v_a / sf * 1.
omega = 0.
total_wind = PointSymWind(sf * 0.5, sf * 0.3, gamma, omega)
def domain(x):
    return bl[0] < x[0] < tr[0] and bl[1] < x[1] < tr[1]

# Time window upper bound
# Estimated through great circle distance + 20 percent
T = 30. * 3600 # 1.2 * geodesic_distance(x_init, x_target, mode='rad') / v_a

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, T=T, coords=coords, autodomain=False, domain=domain)

### Solver parameters

In [4]:
auto_psi = 0.
psi_min = auto_psi - DEG_TO_RAD * 20.
psi_max = auto_psi + DEG_TO_RAD * 20.

nt_pmp = 1000
opti_ceil = 0.01 * sf
neighb_ceil = opti_ceil/2.

solver = Solver(mp,
                x_init,
                x_target,
                T,
                psi_min,
                psi_max,
                output_dir,
                N_disc_init=2,
                opti_ceil=opti_ceil,
                neighb_ceil=neighb_ceil,
                n_min_opti=1,
                adaptive_int_step=False,
                N_iter=nt_pmp)

solver.log_config()
solver.setup()

Solve problem

In [5]:
t_start = time.time()
solver.solve_fancy()
t_end = time.time()
time_pmp = t_end - t_start

Shooting -20.0
Shooting 20.0
Iteration 1 - Shooting 0.0 - Depth 1 - Branching LU
Iteration 2 - Shooting 10.0 - Depth 2 - Branching LU
Iteration 3 - Shooting 15.0 - Depth 3 - Branching LU
Iteration 4 - Shooting 17.5 - Depth 4 - Optimum found
    * Pre-processing :  0.032 s
    * Processing      :  0.062 s
    * Total           :  0.094 s
50436.0


### Explicit control laws

In [6]:
sc = DistanceSC(lambda x: np.linalg.norm(x - x_target), opti_ceil)
sc = TimedSC(T)
from mermoz.feedback import FixedHeadingFB, ConstantFB

mp.load_feedback(FixedHeadingFB(mp.model.wind, v_a, 0., mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(0.))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

Dump everything to output directory

In [7]:
nx = 1001
ny = 1001
mdfm.dump_wind(total_wind, nx=101, ny=101, bl=bl, tr=tr)
mdfm.dump_trajs(mp.trajs)

ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver)
ps.add_param('bl_wind', tuple(bl))
ps.add_param('tr_wind', tuple(tr))
#ps.add_param('nx_wind', nx)
#ps.add_param('ny_wind', ny)
ps.add_param('nt_pmp', nt_pmp)
ps.add_param('pmp_time', time_pmp)
ps.dump()